In [26]:
import os
import pandas as pd
import numpy as np
import glob

In [27]:
path = '/media/DataD800/Alina/retest_set/'

In [28]:
#os.mkdir(path+'MLtables')
path_out = path+'MLtables/'

### 1. Filtering modality tables

In [48]:
#load tables
features = {
    'emo':pd.read_csv(path+'3T_tfMRI_EMOTION_analysis_s2/EMOTION_table_Glasser_FSsubcort.csv', index_col=0),
    'gam':pd.read_csv(path+'3T_tfMRI_GAMBLING_analysis_s2/GAMBLING_table_Glasser_FSsubcort.csv', index_col=0),
    'lan':pd.read_csv(path+'3T_tfMRI_LANGUAGE_analysis_s2/LANGUAGE_table_Glasser_FSsubcort.csv', index_col=0),
    'mot':pd.read_csv(path+'3T_tfMRI_MOTOR_analysis_s2/MOTOR_table_Glasser_FSsubcort.csv', index_col=0),
    'rel':pd.read_csv(path+'3T_tfMRI_RELATIONAL_analysis_s2/RELATIONAL_table_Glasser_FSsubcort.csv', index_col=0),
    'soc':pd.read_csv(path+'3T_tfMRI_SOCIAL_analysis_s2/SOCIAL_table_Glasser_FSsubcort.csv', index_col=0),
    'wm':pd.read_csv(path+'3T_tfMRI_WM_analysis_s2/WM_table_Glasser_FSsubcort.csv', index_col=0),
    'cort':pd.read_csv(path+'3T_Structural_preproc_extended/FS_anatomy_cortical_thickness.csv', index_col=0),
    'subc':pd.read_csv(path+'3T_Structural_preproc_extended/FS_anatomy_subcortical_volume.csv', index_col=0),
    'surf':pd.read_csv(path+'3T_Structural_preproc_extended/FS_anatomy_cortical_area.csv', index_col=0),
    'rest':pd.read_csv(path+'3T_rfMRI_REST_fix/group_rest_FC_z.csv', index_col=0),
    'VolBrain':pd.read_csv(path+'3T_Structural_preproc_extended/totbrainvol_table.csv', index_col=0),
    'movement':pd.read_csv(path+'3T_tfMRI_WM_preproc/all_mods_all_subj_mov.csv', index_col=0)
}

In [49]:
# Drop subjects with missiong values
for key in features.keys():
    features[key] = features[key].dropna(axis=0)

In [50]:
# Check subjects have all modalities
subjects = sorted(list(set(features['emo'].index) & set(features['gam'].index) & set(features['lan'].index) &
set(features['mot'].index) & set(features['rel'].index) & set(features['soc'].index) &
set(features['wm'].index) & set(features['cort'].index) & set(features['subc'].index) &
set(features['surf'].index) & set(features['rest'].index) & set(features['VolBrain'].index) &  set(features['movement'].index)))

print(len(subjects), 'subjects in total')

43 subjects in total


In [51]:
#Filter table according list of subjects having all modalities
for key in features.keys():
    features[key] = features[key].reindex(index=subjects)

In [ ]:
# Special chunk for main set only

#exclude all subjects with A, B and C QS code
df_qs = pd.read_csv(path+'Behavioral_tables/unrestricted.csv', index_col=0)['QC_Issue']
df_qs = df_qs.dropna(axis=0)
name_to_excl1 = []

for ind in df_qs.index:
    if 'A' in df_qs.loc[ind]:
        name_to_excl1+=[ind]
for ind in df_qs.index:
    if 'B' in df_qs.loc[ind]:
        name_to_excl1+=[ind]
#for ind in df_qs.index:
#    if 'C' in df_qs.loc[ind]:
#        name_to_excl1+=[ind]        
        

name_to_excl1 = sorted(set(name_to_excl1))
print(len(name_to_excl1), 'with A, B issue') #or C 
#subjects = [s for s in subjects if s not in name_to_excl1]        
        

#exclude subjects with known major issues from https://wiki.humanconnectome.org/display/PublicData/HCP+Data+Release+Updates%3A+Known+Issues+and+Planned+fixes
i=0
dct_excl={}
for file in glob.glob(path+'QS_issues/*.txt'):
    dct_excl[str(i)] = np.loadtxt(file, dtype=int)
    i+=1
keys = sorted(dct_excl.keys())
name_to_excl2 = dct_excl[keys[0]]
for key in keys[1:]:
    name_to_excl2 = np.append(name_to_excl2, dct_excl[key])

name_to_excl2 = sorted(set(name_to_excl2))
    
print(len(name_to_excl2), 'with major issues')    

#subjects = [s for s in subjects if s not in name_to_excl2]     
subjects_to_exclude = sorted(set(np.append(name_to_excl1, name_to_excl2)))

print(len(subjects_to_exclude), 'subjects need to be excluded')

# Filter subjects
subjects = [s for s in subjects if s not in subjects_to_exclude]
print(len(subjects), 'remains for next retest analysis')

In [52]:
# Special chunk for test-retest set only

#exclude all subjects with A, B and C QS code
df_qs2 = pd.read_csv(path+'Behavioral_tables/unrestricted.csv', index_col=0)['QC_Issue']
df_qs2 = df_qs2.dropna(axis=0)

name_to_excl_1 = []

for ind in df_qs2.index:
    if 'A' in df_qs2.loc[ind]:
        name_to_excl_1+=[ind]
for ind in df_qs2.index:
    if 'B' in df_qs2.loc[ind]:
        name_to_excl_1+=[ind]
#for ind in df_qs2.index:
#    if 'C' in df_qs2.loc[ind]:
#        name_to_excl_1+=[ind] 

name_to_excl_1 = sorted(set(name_to_excl_1))
        
print(len(name_to_excl_1), 'subjects to exclude')
print(name_to_excl_1)

# exclude subjs
subjects = [s for s in subjects if s not in name_to_excl_1]

#check which subjects was used in main set after filtering (write a path to main set file)
#means the main set should be processed first
main_set_sbj_list = sorted(pd.read_csv('/media/DataD800/Alina/MLtables/cognition_table.csv', index_col=0).index)
#and assign them as new list of subjects
#subjects = sorted(set(main_set_sbj_list).intersection(set(subjects)))
subjects = sorted(set(subjects).intersection(set(main_set_sbj_list)))

print(len(subjects), 'remains for next retest analysis')

1 subjects to exclude
[433839]
35 remains for next retest analysis


In [53]:
#Filter table according list of subjects having all modalities
for key in features.keys():
    features[key] = features[key].reindex(index=subjects)
    features[key].to_csv(path_out+str(key)+'_table.csv')

### 2. Creating demographical and cognition tables

In [ ]:
#main set

In [ ]:
#load behavioral tables
beh_tab_s1200 = pd.read_csv(path+'Behavioral_tables/unrestricted.csv', index_col=0)

#choose only cognition columns unadjasted
cog_cols = np.array(beh_tab_s1200.columns[114:166])
cog_cols = [col for col in cog_cols if '_AgeAdj' not in col]
cog_cols = np.append(np.array([cog_cols[-2], cog_cols[-1], cog_cols[-4], cog_cols[-3]], dtype=str), np.array(cog_cols[:-4], dtype=str))

#reindex columns in  table
cog_tab_s1200 = beh_tab_s1200.reindex(index = subjects, columns = cog_cols) #subjects should be from main set
cog_tab_s1200.index.name = None

#save
cog_tab_s1200.to_csv(path_out+'cognition_table.csv')

In [54]:
#retest set

In [55]:
#load behavioral tables
#write a path to the main set behavioral table folder
beh_tab_s1200 = pd.read_csv('/media/DataD800/Alina/Behavioral_tables/unrestricted.csv', index_col=0)
beh_tab_retest = pd.read_csv(path+'Behavioral_tables/unrestricted.csv', index_col=0)

#choose only cognition columns unadjasted
cog_cols = np.array(beh_tab_s1200.columns[114:166])
cog_cols = [col for col in cog_cols if '_AgeAdj' not in col]
cog_cols = np.append(np.array([cog_cols[-2], cog_cols[-1], cog_cols[-4], cog_cols[-3]], dtype=str), np.array(cog_cols[:-4], dtype=str))

#reindex columns in  table
cog_tab_retest = beh_tab_retest.reindex(index = subjects, columns = cog_cols)
cog_tab_retest.index.name = None

#save
cog_tab_retest.to_csv(path_out+'cognition_table.csv')

###### <span style="color:red">Next table with sensitive information you have to have HCP permission for restricted part !</span>

In [56]:
beh_tab_s1200_restr = pd.read_csv('/media/DataD800/Alina/Behavioral_tables/RESTRICTED.csv', index_col=0)

demo_tab_s1200 = pd.DataFrame()
demo_tab_s1200['Gender'] = beh_tab_s1200.loc[subjects, 'Gender']
demo_tab_s1200['Age_in_Yrs'] = beh_tab_s1200_restr.loc[subjects, 'Age_in_Yrs']
demo_tab_s1200['Family_ID'] = beh_tab_s1200_restr.loc[subjects, 'Family_ID']

demo_tab_s1200.to_csv(path_out+'demographics_table.csv')